In [1]:
import pandas as pd
import ast
import requests
import numpy as np

## Merge Two Datasets

This data set was created to list all shows available on Netflix streaming and analyze the data to find interesting facts. It was acquired in July 2022 and contains information available in the United States. However, we still need to verify which shows are Netflix Originals, so the two data sets will be merged.

In [2]:
data = pd.read_csv("netlix_catalogue.csv")

In [3]:
df = pd.read_csv("titles.csv")

In [4]:
# Create a new boolean column by checking if "title" exists in "Name"
df["netflix_owned"] = df["title"].isin(data["Name"])

# Display the updated DataFrame
df.head(2)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,netflix_owned
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN,False
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179,False


In [5]:
# See the amount of movies/tv-shows that are owned(produced) by Netflix
df["netflix_owned"].value_counts()

netflix_owned
False    3816
True     2034
Name: count, dtype: int64

In [6]:
df.to_csv("netflix_data.csv", index=False)

## Data Cleansing

1. Remove titles with no value in them.  
2. Make sure the columns have the correct data types.

In [7]:
df = pd.read_csv("netflix_data.csv")
country_code = pd.read_csv("country_code.csv")

In [8]:
# Check if there's NULL value in the dataset
df.isnull().sum()

# Remove the NULL value in column 'title'
df = df.dropna(subset=["title"])

# Check the data type in columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5849 entries, 0 to 5849
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5849 non-null   object 
 1   title                 5849 non-null   object 
 2   type                  5849 non-null   object 
 3   description           5832 non-null   object 
 4   release_year          5849 non-null   int64  
 5   age_certification     3231 non-null   object 
 6   runtime               5849 non-null   int64  
 7   genres                5849 non-null   object 
 8   production_countries  5849 non-null   object 
 9   seasons               2106 non-null   float64
 10  imdb_id               5446 non-null   object 
 11  imdb_score            5368 non-null   float64
 12  imdb_votes            5352 non-null   float64
 13  tmdb_popularity       5759 non-null   float64
 14  tmdb_score            5539 non-null   float64
 15  netflix_owned         5849

In [9]:
# Capitalize the first letter 
df["type"] = df["type"].str.title()

In [10]:
# Create a new column to show full country name
# Create a dictionary where country code and country name are key and value
code_country = dict(zip(country_code["Code"], country_code["Country"]))

# Make sure value in "production_countries" is a list
df['production_countries'] = df['production_countries'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Write a function to convert country code to country name
def convert(codes):
    result = []
    
    for i in codes:
        result.append(code_country.get(i, i))

    return result

# Create new column to display country name
df["country"] = df["production_countries"].apply(convert)
df.head(3)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,netflix_owned,country
0,ts300399,Five Came Back: The Reference Films,Show,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],[US],1.0,NaN,NaN,NaN,0.600,NaN,False,[America]
1,tm84618,Taxi Driver,Movie,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",[US],NaN,tt0075314,8.2,808582.0,40.965,8.179,False,[America]
2,tm154986,Deliverance,Movie,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",[US],NaN,tt0068473,7.7,107673.0,10.010,7.300,False,[America]


**Request TMDB API to fetch image of movie poster**

In [ ]:
# Assign API key
TMDB_API = "da67f8d56b5921c50d1c55700684de87"

# Define a function to fetch image
def get_image(title, year):
    url = f"https://api.themoviedb.org/3/search/movie?api_key={TMDB_API}&query={title}&year={year}"
    response = requests.get(url).json()
    if response["results"]:
        poster_path = response["results"][0]["poster_path"]
        return f"https://image.tmdb.org/t/p/w500{poster_path}"
    else:
        return None

# Add new column into df dataset
df["poster_url"] = df.apply(lambda row: get_image(row["title"], row["release_year"]), axis=1)

# Create CSV file
df.to_csv("netflix_data_cleaned.csv", index=False)


**Remove invalid poster url**

In [2]:
df = pd.read_csv('netflix_data_cleaned.csv')

In [3]:
# Filter poster_url with invalid link
mask = df["poster_url"].str.contains("https://image.tmdb.org/t/p/w500None", na=False)
df[mask].head(3)

# If the poster_url is invalid, we replace the link with NaN value
df.loc[mask, "poster_url"] = np.nan

# Create CSV file
df.to_csv("netflix_data_cleaned.csv", index=False)